<a href="https://colab.research.google.com/github/fauk74/Machine-Learning/blob/main/Natural_Language_Process_Naive_Bayes_Random_Forest_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('https://raw.githubusercontent.com/fauk74/Dataset/main/Reviews.tsv', delimiter='\t', quoting=3)
#to process the double quotes it will create errors: to ignore we have to add the parameter quoting=3

In [3]:
!wget https://raw.githubusercontent.com/fauk74/Data-Visualization-and-Utility/main/helper_functions.py 


--2021-10-15 18:25:48--  https://raw.githubusercontent.com/fauk74/Data-Visualization-and-Utility/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14292 (14K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  13.96K  --.-KB/s    in 0s      

2021-10-15 18:25:48 (32.3 MB/s) - ‘helper_functions.py’ saved [14292/14292]



In [25]:
from helper_functions import calculate_results

In [5]:
dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
dataset.shape

(1000, 2)

## Cleaning the Text

In [6]:
import re 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')
all_stopwords.remove('no')

corpus= []

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
for i in range(0, len(dataset)):
  review= re.sub('[^a-zA-Z]',' ', dataset['Review'][i])
  review=review.lower()
  review=review.split()
  ps=PorterStemmer()
  review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
  review=' '.join(review)
  corpus.append(review)




In [8]:
import random




In [9]:
#let's try to build a parallel corpus
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')


corpus1=[]
for i in range(0, len(dataset)):
  review=dataset['Review'][i].lower()
  review=tokenizer.tokenize(review)
 # ps=PorterStemmer()
 # review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
  review=' '.join(review)
  corpus1.append(review)


In [10]:
#Let's explore the samples and how the 'cleaning' affected the text
a=random.randint(0,1000)
print(f"Sentence n. {a}")
print(dataset.iloc[a][0])
print('\n')
print(f"Corpus baseline \n{corpus[a]}")
print('\n')
print(f"Corpus 1\n{corpus1[a]}")
print("\n")


Sentence n. 997
Overall I was not impressed and would not go back.


Corpus baseline 
overal not impress would not go back


Corpus 1
overall i was not impressed and would not go back




# Creating the Bag of Words Model

In [11]:
#Corpus is still in words; X is CountVectorized ; X1 is Corpu with stopwords
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X = cv.fit_transform(corpus).toarray()
X1 = cv.fit_transform(corpus1).toarray()



In [146]:
len(X[0])

1567

In [147]:
len(X1[0])

2035

In [13]:
len(corpus)

1000

In [17]:
corpus[:10]

['wow love place',
 'crust not good',
 'not tasti textur nasti',
 'stop late may bank holiday rick steve recommend love',
 'select menu great price',
 'get angri want damn pho',
 'honeslti tast fresh',
 'potato like rubber could tell made ahead time kept warmer',
 'fri great',
 'great touch']

In [19]:
y=dataset['Liked']

In [155]:
#The vectorization is done before the split 

In [20]:

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)



In [168]:
# Function to evaluate: accuracy, precision, recall, f1-score



# Model 0: Naive Bayes Multinomial

In [21]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([

                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [22]:
y_pred=model_0.predict(X_test)

In [23]:
model_0_score = model_0.score(X_test, y_test)
print(f"Our baseline model Naive Bayes Multinomial achieves an accuracy of: {model_0_score*100:.2f}%")

Our baseline model Naive Bayes Multinomial achieves an accuracy of: 80.00%


In [26]:
model_0_results=calculate_results(y_true=y_test, 
                                  y_pred=y_pred)
cm_0=confusion_matrix(y_test, y_pred)
model_0_results

{'accuracy': 80.0, 'f1': 0.8, 'precision': 0.8, 'recall': 0.8}

In [27]:
cm_0

array([[77, 20],
       [20, 83]])

# Model 1: Naive Bayes Gaussian

In [28]:

# Create tokenization and modelling pipeline
model_1 = Pipeline([
                    ("clf", GaussianNB()) # model the text
])
# Fit the pipeline to the training data
model_1.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('clf', GaussianNB(priors=None, var_smoothing=1e-09))],
         verbose=False)

In [29]:
model_1_score = model_1.score(X_test, y_test)
print(f"Our  model Naive Bayes Gaussian achieves an accuracy of: {model_1_score*100:.2f}%")

Our  model Naive Bayes Gaussian achieves an accuracy of: 73.00%


In [30]:
y_pred=model_1.predict(X_test)
model_1_results=calculate_results(y_true=y_test, 
                                  y_pred=y_pred)
cm_1=confusion_matrix(y_test, y_pred)
model_1_results

{'accuracy': 73.0,
 'f1': 0.722465894997933,
 'precision': 0.7505027494108405,
 'recall': 0.73}

In [31]:
cm_1

array([[55, 42],
       [12, 91]])

In [ ]:
#We have to Try 

#CART

#C5.0

#Maximum Entropy

# Model 2: Naive Bayes Multinomial with Corpus stopwords not removed

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size = 0.2, random_state = 0)

In [33]:
# Create  modelling pipeline
model_2 = Pipeline([
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_2.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [34]:
y_pred=model_2.predict(X_test)
model_2_results=calculate_results(y_true=y_test, 
                                  y_pred=y_pred)
cm_2=confusion_matrix(y_test, y_pred)
model_2_results

{'accuracy': 80.5,
 'f1': 0.8050438848740967,
 'precision': 0.80545,
 'recall': 0.805}

In [ ]:
#We can see that not removing the stopwords we get up to 80,5% of F1 

#  Model 3 Tfidvectorizer + Multinomial Naive Bayes + Corpus stopwrods not removed

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer



In [41]:
# Create tokenization and modelling pipeline
model_3 = Pipeline([
                   ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])


In [45]:
X_train, X_test, y_train, y_test = train_test_split(corpus1, y, test_size = 0.2, random_state = 0)

In [46]:
model_3.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [47]:
y_pred=model_3.predict(X_test)
model_3_results=calculate_results(y_true=y_test, 
                                  y_pred=y_pred)
cm_3=confusion_matrix(y_test, y_pred)
model_3_results

{'accuracy': 79.0,
 'f1': 0.7900420168067228,
 'precision': 0.7902420242024203,
 'recall': 0.79}

# Model 4 Tfidvectorizer + Multinomial Naive Bayes + Corpus stopwords removed

In [229]:
# Create tokenization and modelling pipeline
model_4 = Pipeline([
                   ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])


In [230]:
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size = 0.2, random_state = 0)

In [231]:
model_4.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [232]:
y_pred=model_4.predict(X_test)
model_4_results=calculate_results(y_true=y_test, 
                                  y_pred=y_pred)
cm_4=confusion_matrix(y_test, y_pred)
model_4_results

{'accuracy': 80.0,
 'f1': 0.8000400160064026,
 'precision': 0.8002400240024002,
 'recall': 0.8}

# Model 5: Random Forest Classifier

In [61]:
from sklearn.ensemble import RandomForestClassifier
model_5 = RandomForestClassifier(n_estimators = 1000, criterion = 'entropy', random_state = 0)




X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size = 0.2, random_state = 0)


model_5.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [62]:
y_pred=model_5.predict(X_test)
model_5_results=calculate_results(y_true=y_test, 
                                  y_pred=y_pred)
cm_5=confusion_matrix(y_test, y_pred)
model_5_results

{'accuracy': 81.5,
 'f1': 0.8147454659161977,
 'precision': 0.8194504830917875,
 'recall': 0.815}

# Model 6: XGBoost

In [79]:
from xgboost import XGBClassifier

In [82]:
model_6=XGBClassifier(n_estimators=1000)
model_6.fit(X_train, y_train)

y_pred=model_6.predict(X_test)
model_6_results=calculate_results(y_true=y_test, 
                                  y_pred=y_pred)
cm_6=confusion_matrix(y_test, y_pred)
model_6_results

{'accuracy': 78.5,
 'f1': 0.7847041901188243,
 'precision': 0.7891847826086956,
 'recall': 0.785}